In [46]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_PATH = Path('../csv_files/raw')
INGREDIENT = 'ingredient.csv'
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row', None)

# 1. (비선호음식, 원재료명)

In [ ]:
ingredient_df = pd.read_csv(DATA_PATH / INGREDIENT, encoding="utf-8")
print(f'shape: {ingredient_df.shape}')
ingredient_df.head(5)

In [ ]:
def search(pname, exclude):
   if exclude:
      filtered = ingredient_df[ingredient_df['기존'].str.contains(non)==False]
      filtered = filtered[filtered['기존'].str.contains(pname)==True]
      print(f'{filtered.shape}')
      display(filtered)
      return

   filtered = ingredient_df[ingredient_df['기존'].str.contains(pname)==True]
   print(f'{filtered.shape}')
   display(filtered)
      

In [ ]:
# 검색 키워드: pname="게"
# 제외할 키워드: exclude="게맛살|게장"
search(pname="게", exclude=None)

# 2. (haccp의 원재료 데이터, 비선호음식)
1. (품목보고번호, 비선호음식)이 연결된 테이블 만들기
2. (품목보고번호, 원재료 string) 

### 2-1. (품목보고번호, 비선호음식)이 연결된 테이블 만들기

#### haccp

In [203]:
# 데이터 디렉토리 설정
HACCP = "HACCP.csv"
DISLIKED = "disliked_to_rawmtr.csv"

In [204]:
# haccp dataframe 열기
haccp_df = pd.read_csv(DATA_PATH / HACCP, encoding="utf-8")
print(f'{haccp_df.shape}')
haccp_df = haccp_df.reset_index().rename(columns={"index": "id"})
haccp_df = haccp_df.drop(columns=['item.barcode', 'item.allergy'])
haccp_df.head(5)

(15002, 5)


,id,item.prdlstNm,item.rawmtrl,item.prdlstReportNo
0,0,설화눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",201904962715
1,1,설화눈꽃팝김부각스낵 아몬드맛,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",201904962712
2,2,눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",201904962711
3,3,참군고구마칩,군고구마,201903787252
4,4,참군고구마,군고구마,201903787251


In [205]:
# rawmtrs 다른 행으로 분리
haccp_df['item.rawmtrl'] = haccp_df['item.rawmtrl'].str.split(',')
haccp_df_exploded = haccp_df.explode('item.rawmtrl')
haccp_df_exploded.head(5)

,id,item.prdlstNm,item.rawmtrl,item.prdlstReportNo
0,0,설화눈꽃팝김부각스낵,찹쌀,201904962715
0,0,설화눈꽃팝김부각스낵,김,201904962715
0,0,설화눈꽃팝김부각스낵,참깨,201904962715
0,0,설화눈꽃팝김부각스낵,옥수수기름(옥배유),201904962715
0,0,설화눈꽃팝김부각스낵,양파,201904962715


#### disliked

In [206]:
disliked_df = pd.read_csv(DATA_PATH / DISLIKED, encoding="utf-8")
disliked_df = disliked_df.reset_index().rename(columns={"index": "id"})
print(f'{disliked_df.shape}')
disliked_df.head(5)

(21, 3)


,id,disliked,rawmtrs
0,0,우유/버터/치즈,"분유, 우유, 유청, 밀크, 밀키, 밀크레진, 낙우밀"
1,1,계란,"달걀, 계란, 난류, 난백, 난황, 전란, 유정란, 난각, 마요네즈, 에그, 배터믹스-EL, 배터믹스-이엘"
2,2,새우/게,"새우, 꽃게, 대게, 게맛살, 게 엑기스, 굴소스(게), 게농축액, 갑각류, 다시마게"
3,3,글루텐,"소맥분, 소맥, 밀가루, 부침가루, 빵가루, 글루텐, 밀식이섬유, 배터용분말, 치킨용배터믹스, 지미베이스, 가라아게배터, 통밀, 찐밀쌀가루, 호밀, 밀베이커스믹스, 중력분, 조미새우분말, 그릴시즈닝-22, (밀), 소밀, 밀분해액, 전분, 보리, 찰보리, 귀리"
4,4,메밀,메밀


In [207]:
# rawmtrs 다른 행으로 분리
disliked_df['rawmtrs'] = disliked_df['rawmtrs'].str.split(', ')
disliked_df_exploded = disliked_df.explode('rawmtrs')
disliked_df_exploded.head(5)

,id,disliked,rawmtrs
0,0,우유/버터/치즈,분유
0,0,우유/버터/치즈,우유
0,0,우유/버터/치즈,유청
0,0,우유/버터/치즈,밀크
0,0,우유/버터/치즈,밀키


### merge

In [214]:
merged_df = pd.merge(haccp_df_exploded, disliked_df_exploded, how='cross')
print(f'{merged_df.shape}')

(45307872, 7)


In [215]:
def merge_function(row):
    try:
        return  row['rawmtrs'] in row['item.rawmtrl']
    except:
        return False

In [216]:
result = merged_df[merged_df.apply(merge_function, axis=1)]
print(f'{result.shape}')
result.head(5)

(306506, 7)


,id_x,item.prdlstNm,item.rawmtrl,item.prdlstReportNo,id_y,disliked,rawmtrs
85,0,설화눈꽃팝김부각스낵,찹쌀,201904962715,7,소,
268,0,설화눈꽃팝김부각스낵,김,201904962715,7,소,
451,0,설화눈꽃팝김부각스낵,참깨,201904962715,7,소,
634,0,설화눈꽃팝김부각스낵,옥수수기름(옥배유),201904962715,7,소,
817,0,설화눈꽃팝김부각스낵,양파,201904962715,7,소,


In [225]:
result_nospace = result[result['rawmtrs']!='']
print(f'{result_nospace.shape}')

(58922, 7)


,id_x,item.prdlstNm,item.rawmtrl,item.prdlstReportNo,id_y,disliked,rawmtrs
1666,0,설화눈꽃팝김부각스낵,새우,201904962715,2,새우/게,새우
2378,0,설화눈꽃팝김부각스낵,건표고버섯,201904962715,20,버섯,버섯
2438,0,설화눈꽃팝김부각스낵,둥굴레,201904962715,6,오징어/어패류,굴
3778,1,설화눈꽃팝김부각스낵 아몬드맛,아몬드,201904962712,11,잣/호두/아몬드,아몬드
4777,1,설화눈꽃팝김부각스낵 아몬드맛,새우,201904962712,2,새우/게,새우
5489,1,설화눈꽃팝김부각스낵 아몬드맛,건표고버섯,201904962712,20,버섯,버섯
5549,1,설화눈꽃팝김부각스낵 아몬드맛,둥굴레,201904962712,6,오징어/어패류,굴
7705,2,눈꽃팝김부각스낵,새우,201904962711,2,새우/게,새우
8417,2,눈꽃팝김부각스낵,건표고버섯,201904962711,20,버섯,버섯
8477,2,눈꽃팝김부각스낵,둥굴레,201904962711,6,오징어/어패류,굴


In [226]:
result_nospace = result_nospace.drop(columns=['id_x', 'id_y'])
result_nospace.head(10)

,item.prdlstNm,item.rawmtrl,item.prdlstReportNo,disliked,rawmtrs
1666,설화눈꽃팝김부각스낵,새우,201904962715,새우/게,새우
2378,설화눈꽃팝김부각스낵,건표고버섯,201904962715,버섯,버섯
2438,설화눈꽃팝김부각스낵,둥굴레,201904962715,오징어/어패류,굴
3778,설화눈꽃팝김부각스낵 아몬드맛,아몬드,201904962712,잣/호두/아몬드,아몬드
4777,설화눈꽃팝김부각스낵 아몬드맛,새우,201904962712,새우/게,새우
5489,설화눈꽃팝김부각스낵 아몬드맛,건표고버섯,201904962712,버섯,버섯
5549,설화눈꽃팝김부각스낵 아몬드맛,둥굴레,201904962712,오징어/어패류,굴
7705,눈꽃팝김부각스낵,새우,201904962711,새우/게,새우
8417,눈꽃팝김부각스낵,건표고버섯,201904962711,버섯,버섯
8477,눈꽃팝김부각스낵,둥굴레,201904962711,오징어/어패류,굴


In [ ]:
result_nospace.to_csv(DATA_PATH / "product_to_rawmtr.csv")